In [74]:
import pickle

import requests
from dateutil.parser import parse
from jira import JIRA
from pymongo import MongoClient

%matplotlib inline
from flask import Flask
from flask import jsonify

from pymongo import MongoClient
import pandas as pd
from datetime import datetime, timedelta
from collections import Counter
from statistics import mean
from itertools import groupby


import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import numpy as np
import itertools
import warnings

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error

import requests

In [75]:
client = MongoClient('localhost:27017')
db = client.poc_teradata

In [84]:
mg_boards = db.boards

board_ids = [board['_id'] for board in mg_boards.find({})]

In [77]:
board_project_dict = {
    513: 'TDMC',
    491:'VED',
    419:'TAWS',
    600: 'TAZURE'
}

In [78]:
data = {
        'jql': "project in ('TDMC') and (type=larva OR type=bug)"
    }

url = "https://jira.td.teradata.com/jira/rest/api/2/search?jql=project in ('TDMC') and (type=larva OR type=bug)"

response = requests.get(url, auth=('aw186034', 'GRAY5sky')).json()

In [79]:
def get_response(project_name):
    print(project_name)

    url = f"https://jira.td.teradata.com/jira/rest/api/2/search?jql=project in ('{project_name}') and (type=larva OR type=bug)"

    response = requests.get(url, auth=('aw186034', 'GRAY5sky')).json()
    return response

In [80]:
def get_issues(issues, board_id):
    result = []
    for issue in issues:
        key = issue['key']
        id = issue['id']
        assignee_dict = issue['fields']['assignee']
        status = issue['fields']['status']['name']
        issue_type = issue['fields']['issuetype']['name']
        created = parse(issue['fields']['created'])
        issue_dict = {'key': key, '_id': id, 'board_id': board_id, 'assignee' : assignee_dict, 'status':status, 'issueType': issue_type, 'created' : created}
        result.append(issue_dict)
    return result

In [81]:
result = []

for board, project in board_project_dict.items():
    response = get_response(project)
    issues = get_issues(response['issues'], board)
    result.extend(issues)

TDMC


VED


TAWS


TAZURE


In [82]:
len(result)

196

In [83]:
mg_issues = db.issues

ids = mg_issues.insert_many(result)

In [93]:
board_id = 600
issues = list(mg_issues.find({'board_id': board_id}, {'_id': 0, 'created': 1, 'issueType': 1}))


for issue in issues:
    issue['created'] = issue['created'].strftime("%Y%m%d")

In [103]:
sorted_issues = sorted(issues, key=lambda k: k['created'])

result = {}
prev_larva = 0
prev_bug = 0
for issue in sorted_issues:
    date = issue['created']
    type = issue['issueType']
    if type == 'Bug':
        prev_bug += 1
    elif type == 'Larva':
        prev_larva += 1
    result[date] = {'larva': prev_larva, 'bug': prev_bug}

In [105]:
result = []

for date, value in result_dict.items():
    response_dict = {'date':date, 'Bug':value['bug'], 'Larva':value['larva']}
    result.append(response_dict)
    
result

[{'Bug': 1, 'Larva': 0, 'date': '20161219'},
 {'Bug': 2, 'Larva': 0, 'date': '20161220'},
 {'Bug': 3, 'Larva': 0, 'date': '20161221'},
 {'Bug': 5, 'Larva': 0, 'date': '20161222'},
 {'Bug': 6, 'Larva': 0, 'date': '20161224'},
 {'Bug': 7, 'Larva': 0, 'date': '20161228'},
 {'Bug': 8, 'Larva': 0, 'date': '20170103'},
 {'Bug': 9, 'Larva': 0, 'date': '20170105'},
 {'Bug': 11, 'Larva': 0, 'date': '20170106'},
 {'Bug': 12, 'Larva': 0, 'date': '20170118'},
 {'Bug': 13, 'Larva': 0, 'date': '20170124'},
 {'Bug': 14, 'Larva': 0, 'date': '20170126'},
 {'Bug': 15, 'Larva': 0, 'date': '20170127'},
 {'Bug': 16, 'Larva': 0, 'date': '20170220'},
 {'Bug': 17, 'Larva': 0, 'date': '20170306'},
 {'Bug': 18, 'Larva': 0, 'date': '20170313'},
 {'Bug': 19, 'Larva': 0, 'date': '20170316'},
 {'Bug': 20, 'Larva': 0, 'date': '20170328'},
 {'Bug': 21, 'Larva': 0, 'date': '20170331'},
 {'Bug': 22, 'Larva': 0, 'date': '20170404'},
 {'Bug': 25, 'Larva': 0, 'date': '20170427'},
 {'Bug': 26, 'Larva': 0, 'date': '20170518